Dou et al. (2020) model

Agora em Julia versão 1.5.3

This script simulates the model

Calculando a derivada fixando os parâmetros iguais aos de Dou et al. e substituindo pelo $\beta$ encontrado na última estimação com 2 clusters

In [1]:
using LinearAlgebra, Statistics

using DataFrames
using BenchmarkTools


In [2]:
using Distributed
nprocs = 2
addprocs(nprocs);

@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

#importou a função solve_tree

pids = workers()
aux = pids[nprocs]

3

In [3]:
#Vh/D, L/D, Dj/D
data1 = [0.86, 0.19, 0.8];
n1 = 49

data2 = [2.65, 0.69, 0.74];
n2 = 17

#ρ, β, c0, λj
game_parameters = [0.93, 9.48, 0.09, 0.96];

In [4]:
A2 = @spawnat aux solve_tree(data2, game_parameters)

A1 = solve_tree(data1, game_parameters);

A2 = fetch(A2);

205.274318 seconds (634.66 M allocations: 14.096 GiB      From worker 3:	189.409930 seconds (576.80 M allocations: 12.816 GiB, 1.22% gc time)
, 1.22% gc time)


Simulation parameters

In [5]:
#θs0, θj0, μ
simulation_parameters = [49.0, 30.0, 3.23];

### Criando funções para simular o jogo

In [6]:
#cost function
function Ct(t, c0=c0, c1=c1)
    
    #didn't use D because is normalized to D == 1.0
    #cost at period t=0(index1) is 0
    if(t <= 1)
        return 0
    else
        return c0 + c1 * (t-1) #test to make index==1 be t==0
    end
end



#liquidation payoffs
function s_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1), Ds)
end


function j_L(t, L, Ds, Dj, c0, c1)
    return min(L - Ct(t, c0, c1) - s_L(t, L, Ds, Dj, c0, c1), Dj)
end


function proposal(Pkt_array, t, hkt, lmt)
    
    #pkt array será sempre do propositor, quem responder às propostas apenas olhará o seu valor de continuação
    
    return policy, payoff_prop, payment, lm_next = Pkt_array[t, hkt, lmt, [end,end-1, 1, 2]]
    
end
    

#maximum value of reorganization each period
function Vt(Vmax, ρ, t)

    if(t <=1)
        return Vmax
    else
        #(t-2) instead of (t-1) because we shifted the indexes in the game so as to include t==0 at index==1
        return ρ^(t-2) * Vmax
    end


end

# answer_liq
function answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
    
    liq = (m_L(t, L, Ds, Dj, c0, c1), Vt(Vmax, ρ, t) * hm_next/100 - Ct(t, c0, c1) - payoff_prop)
    
    payoff_liq, answer = findmax(liq)
    
    return payoff_liq, answer
end

function answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)
    
    reorg_value = (payment, Cont_val[t+1, hm_next, lm_next, lk_next])

    payoff_reorg, answer = findmax(reorg_value)
    
    return payoff_reorg, answer
end

function update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
    
    if(propositor=="s")
        hst = hk_next
        lst = lk_next
        
        hjt = hm_next
        ljt = lm_next
        
    else
        hst = hm_next
        lst = lm_next
        
        hjt = hk_next
        ljt = lk_next
    end
    
    
    return hst, lst, hjt, ljt
end

        

update_beliefs (generic function with 1 method)

In [7]:
#código para tirar um draw da beta

function draw_beta(hkt, β=game_parameters[2], grid=100)

    u = rand()

    if(hkt == grid)
        
        return grid

    else
        
        x = 1.0 - exp(1.0/β * (log(1.0 - u) + β * log(1.0 - hkt/grid)))
        x = round(x * 100, digits=0)
#         return Int64(x * 100) #to convert in an integer
        return Int64(x)
        
    end
end

        

# @code_warntype draw_beta(10)



draw_beta (generic function with 3 methods)

In [8]:
function choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
    
    if(propositor=="s")
        
        Pkt_array = Pst_array
        Cont_val = j_W
        prop_index = 1
        respondent_index = 2 #índice de j, para organizar o payoff
        m_L = j_L

        hkt = hst
        lkt = lst
        
        hmt = hjt
        lmt = ljt


        lk_next = hkt

        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)
        
    elseif(propositor=="j")
        Pkt_array = Pjt_array
        Cont_val = s_W
        prop_index = 2
        respondent_index = 1
        m_L = s_L

        hkt = hjt
        lkt = ljt
        
        hmt = hst
        lmt = lst


        lk_next = hkt


        hk_next = draw_beta(hkt)
        hm_next = draw_beta(hmt)

    else
        
        println("error: propositor not valid")
        
    end
    
    return Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next
end




# #test
# Hs0 = 30
# Hj0 = 28


# hst = Hs0
# hjt = Hj0

# #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
# lst = hst
# ljt = hjt




# choose_parameters("s", hst, lst, hjt, ljt);
    
    

In [9]:
function simulate_game(data, game_parameters, simulation_parameters, s_W=s_W, j_W=j_W, Pst_array=Pst_array, Pjt_array=Pjt_array, t=1, grid=100)
    
    
    Vmax = data[1]
    L = data[2]
    Dj = data[3]
    Ds = one(1.0) - Dj
    
    
    c0 = game_parameters[3];
    c1 = c0/30;

    ρ = game_parameters[1];
    λj = game_parameters[4];

    
    
#     hst = Hs0
#     hjt = Hj0
    
    hst = Int64(simulation_parameters[1])
    hjt = Int64(simulation_parameters[2])

    #assumindo que lower bounds nos períodos iniciais são as próprias habilidades iniciais
    lst = hst
    ljt = hjt
    
    result = zeros(5);
    
    #number of observed proposals
    observed_proposals = 0.0
    
    
    #recovering "T+1" from s_W 
    T = size(s_W,1)
    
    
    while(result[1]==0.0 && t < T)
        
        
        u = rand()

        if(u < λj)
            propositor = "j"

        else
            propositor = "s"

        end

        
        #setting the default variables according to the propositor
        Pkt_array, Cont_val, prop_index, respondent_index, m_L, hkt, lkt, hmt, lmt, lk_next, hk_next, hm_next = choose_parameters(propositor, hst, lst, hjt, ljt, Pst_array, Pjt_array, s_W, j_W)
        
        #proposal ####
        policy, payoff_prop, payment, lm_next = proposal(Pkt_array, t, hkt, lmt)

        #lm_next is the update of the adversary's lower bound
        if(lm_next>=grid+1 || policy!= 3.0) #para evitar update de lowerbound quando proponente não propõe reorg
            lm_next = lmt
        else
            lm_next = Int64(lm_next)
        end
        
        
        
        if(policy==3.0)
            
            observed_proposals += 1.0
    
            payoff_respondent, answer = answer_reorg(payment, Cont_val, t, hm_next, lm_next, lk_next)


            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals

            else
                
                t+=1

                hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            end

        elseif(policy==2.0)
            
            t+=1
            
            hst, lst, hjt, ljt = update_beliefs(propositor, hk_next, hm_next, lk_next, lm_next)
            
            

        else      
            #(policy==1.0)
            
            observed_proposals += 1.0

            payoff_respondent, answer = answer_liq(m_L, t, hm_next, payoff_prop, L, Ds, Dj, Vmax, ρ, c0, c1)
            
            

            if(answer==1.0)

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 1.0
                result[4] = t
                result[5] = observed_proposals

            else

                result[prop_index] = payoff_prop
                result[respondent_index] = payoff_respondent
                result[3] = 3.0
                result[4] = t
                result[5] = observed_proposals


            end
        end


        if(t==T)
            
            result[1] = s_W[T, 1, 1, 1]#todos os valores finais de S são iguais, então acessarei o índice 1
            result[2] = j_W[T, 1, 1, 1]
            result[3] = 1.0
            result[4] = t
            result[5] = max(observed_proposals, 1.0)
            
        end
        




    end
    
    
    return result
end

        
        
#precisa colocar uma condição IF aqui para caso o jogo vá até o último período?
    

simulate_game (generic function with 7 methods)

In [10]:
#unit testing
@btime simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])

  3.200 μs (21 allocations: 947 bytes)


5-element Array{Float64,1}:
 0.06619479127656211
 0.2026410958782336
 3.0
 8.0
 5.0

In [11]:
function loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    
    
    
    #S is the number of simulations
    #N is the number of observations
    #data1 is the center of the first cluster
    #A1 is the collection of arrays from the first cluster (s_W, j_W, Pst, Pjt)
    #n1 is the number of observations in the first cluster
    
    
    
    
    #no futuro os argumentos podem ser arrays de arrays, assim ele faz o loop para cada cluster
    Results = zeros(S, N, 7)

    #últimas entradas de Results são os valores das dívidas
    
    
    for s in 1:S
        for n in 1:N
            if(n <= n1)
                Results[s,n,1:5] .= simulate_game(data1, game_parameters, simulation_parameters, A1[1], A1[2], A1[3], A1[4])
                Results[s,n, end-1] = A1[5]
                Results[s,n, end] = A1[6]
            else
                Results[s,n,1:5] .= simulate_game(data2, game_parameters, simulation_parameters, A2[1], A2[2], A2[3], A2[4])
                Results[s,n, end-1] = A2[5]
                Results[s,n, end] = A2[6]
            end
            
        end
    end

    return Results
    
end

    
# loop_simulations(40, 75, θs0, θj0, μ, λj);

In [12]:
#unit testing
S = 40
N = 66
Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

40×66×7 Array{Float64,3}:
[:, :, 1] =
 0.069264   0.0792672  0.0609115  0.0792672  …  0.688149  0.397665  0.537844
 0.0711663  0.0722816  0.0716256  0.0560481     0.802939  0.476464  0.350157
 0.0729475  0.0738825  0.191478   0.191478      0.381256  0.504366  0.749646
 0.0668676  0.0792672  0.0632119  0.0667097     0.57873   0.381954  0.491193
 0.101875   0.0696205  0.0709448  0.0699501     0.452663  0.363727  0.351775
 0.06909    0.0576009  0.0677254  0.0750766  …  0.372766  0.392974  0.405668
 0.0607452  0.0618983  0.0696844  0.0746116     0.354104  0.376323  0.381516
 0.0693102  0.0745803  0.0792672  0.0716095     0.382428  0.573276  0.383512
 0.0694015  0.191478   0.0653573  0.0728415     0.355357  0.362929  0.371937
 0.0625352  0.0616535  0.0639049  0.124145      0.377513  0.383677  0.384687
 0.132237   0.0615039  0.0694767  0.0652112  …  0.386804  0.404149  0.372389
 0.185274   0.0694629  0.0665653  0.0654785     0.334919  0.382588  0.356828
 0.0603346  0.0675425  0.0792672  0.06

In [13]:
function simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    Results = loop_simulations(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    Moments = zeros(S, 8)
    
    μ = simulation_parameters[3]
    for s in 1:S

            DF = DataFrame()
            DF.payoff_s = Results[s,:,1];
            DF.payoff_j = Results[s,:,2];
            DF.out = Results[s,:,3];
            DF.t = Results[s,:,4];
            DF.observed_proposals = Results[s,:,5];
            DF.Ds = Results[s,:,6];
            DF.Dj = Results[s,:,7];


            DF.outcome = ifelse.(DF.out .==3.0, "R", "L");


            #desfazendo o deslocamento no índice de t
            DF.t = DF.t .- 1.0;

            # DF.incourt = ifelse.(DF.t .> 0.0, "incourt", "precourt");

            INCOURT = filter(DF -> DF.t .> 0.0, DF);
            PRECOURT = filter(DF -> DF.t .<= 0.0, DF);

            PRECOURT_REORG = filter(PRECOURT -> PRECOURT.outcome .== "R", PRECOURT)
            INCOURT_REORG = filter(INCOURT -> INCOURT.outcome .== "R", INCOURT)

            #cálculo dos momentos
            #1. avg log number of months between observed proposals incourt

            mm1 = INCOURT
            mm1 = log.(mm1.t .* μ ./ mm1.observed_proposals)
            mm1 = mean(mm1)

            #2. fraction reorganized given that the case went into court

            mm2 = size(INCOURT_REORG,1)/ size(INCOURT,1)

            #3. ln duration of court cases in months

            mm3 = INCOURT
            mm3 = filter(mm3 -> mm3.t .> 0.0, mm3) #removendo os casos 0 para não poluir a média
            mm3.t = mm3.t .* μ
            mm3 = mean(log.(mm3.t)) #log here uses exp as base, so it's the same as ln

            #4. fraction of cases incourt
            mm4 = size(INCOURT,1) / size(DF, 1)

            #5. avg recovery rate for senior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm5 = PRECOURT_REORG
            mm5.R_s = mm5.payoff_s ./ mm5.Ds
            mm5 = mean(mm5.R_s)



            #6. avg recovery rate for junior given precourt REORGANIZATION

            #aqui nós temos Ds, então é mais fácil. Preciso automatizar isso no código quando for tudo escalado

            mm6 = PRECOURT_REORG
            mm6.R_j = mm6.payoff_j ./ mm6.Dj
            mm6 = mean(mm6.R_j)


            #7. junior avg fraction gain given incourt REORGANIZATION

            mm7 = INCOURT_REORG
            mm7 = mean(mm7.payoff_j ./ (mm7.payoff_j .+ mm7.payoff_s )) 

            #8. total recovery rate given incourt REORGANIZATION

            mm8 = INCOURT_REORG
            mm8 = mean( mm8.payoff_s .+ mm8.payoff_j)
        
        
            Moments[s,:] .= [mm1, mm2, mm3, mm4, mm5, mm6, mm7, mm8]

            
            #if there are no INCOURT or no PRECOURT cases, it will return NaN
            #thus, we replace it by zero
            Moments[s,:] .= replace!(Moments[s,:], NaN=> zero(0.0))

        end
    return vec(mean(Moments, dims=1))
end



  

simulate_moments (generic function with 1 method)

In [14]:
using Random

Random.seed!(2021)

#test
S = 100
N = 66

simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

8-element Array{Float64,1}:
 1.5271228828532797
 1.0
 2.7495501630793773
 0.9010606060606059
 0.6844557550763442
 0.3599257326897818
 0.7092836267924288
 0.5393246108532963

Calculando a matriz de pesos ótima com os resultados dessa estimação


Two-step variance covariance estimator of W
https://notes.quantecon.org/submission/5b3db2ceb9eab00015b89f93



In [15]:
#data_moments
data_moments = [1.479, 0.843, 2.76, 0.969, 0.6235, 0.5569, 0.773, 0.497];

In [16]:
function omega(N, data_moments, simulated_moments)
    
    error = (simulated_moments .- data_moments)
    
    omega = 1/N * error * transpose(error)
    
    return omega
end

omega (generic function with 1 method)

In [21]:
W_2step = pinv(omega(N, data_moments, simulated_moments))

8×8 Array{Float64,2}:
  23.8488     77.8062   -5.17874  …   -97.6167   -31.5766    20.9753
  77.8062    253.841   -16.8955      -318.473   -103.018     68.4314
  -5.17874   -16.8955    1.12456       21.1974     6.85683   -4.55476
 -33.6695   -109.846     7.31129      137.814     44.5795   -29.6127
  30.2085     98.5547   -6.55975     -123.648    -39.9971    26.5687
 -97.6167   -318.473    21.1974   …   399.56     129.248    -85.8549
 -31.5766   -103.018     6.85683      129.248     41.8085   -27.772
  20.9753     68.4314   -4.55476      -85.8549   -27.772     18.448

In [22]:
#pesos dados a cada um dos momentos
diag(W_2step)

8-element Array{Float64,1}:
  23.848775419922433
 253.84120229035938
   1.1245579422256526
  47.53419550058367
  38.26416512321691
 399.55981066613015
  41.808530787292675
  18.44797743628384

Calculando a derivada dos momentos em relação aos parâmetros

In [16]:
function simulate_new_moments(i, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)
    
    
    println("parameter ",i, game_parameters, simulation_parameters)
        
    θs0, θj0, μ = simulation_parameters;
    θs0 = Int64(round(θs0, digits=0));
    θj0 = Int64(round(θj0, digits=0));

    λj = game_parameters[4];
    new_simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1) 
    
    return new_simulated_moments
end

simulate_new_moments (generic function with 1 method)

In [22]:
 
function derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

    
    
    
    #https://en.wikibooks.org/wiki/Introduction_to_Numerical_Methods/Numerical_Differentiation
    #backward divided difference method

   


    estimated_game_parameters = game_parameters
    estimated_simulation_parameters = simulation_parameters



    A2 = @spawnat aux solve_tree(data2, game_parameters)
    A1 = solve_tree(data1, game_parameters)
    A2 = fetch(A2)
    
    
    
    simulated_moments = simulate_moments(S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

    P = size(game_parameters,1)+size(simulation_parameters,1)
    M = size(simulated_moments,1)
    D = zeros(M, P)
    
    #porcentagem de variação no parâmetro
    pct = 1/100
    
    for p in P:-1:1

        if(p>size(game_parameters,1))

            simulation_parameters = copy(estimated_simulation_parameters)
            
            delta = simulation_parameters[p-size(game_parameters,1)] * pct

            simulation_parameters[p-size(game_parameters,1)] -= delta


            if(p == 5 || p ==6) #para ajustar caso seja habilidade inicial de S ou de J
                simulation_parameters[p-size(game_parameters,1)] = round(simulation_parameters[p-size(game_parameters,1)], digits=0)
            end
            
            
            #caso a variação percentual não seja suficiente para reduzir habilidades iniciais, delta será 1
            if(simulation_parameters[p-size(game_parameters,1)] == estimated_simulation_parameters[p-size(game_parameters,1)])
                delta = Int64(1)
                
                simulation_parameters[p-size(game_parameters,1)] -= delta
            end

            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta
            
        else

            #se for um parâmetro de jogo, precisa recalcular o jogo

            game_parameters = copy(estimated_game_parameters)
            
            delta = game_parameters[p] * pct
            game_parameters[p] -= delta

            A2 = @spawnat aux solve_tree(data2, game_parameters)
            A1 = solve_tree(data1, game_parameters)
            A2 = fetch(A2)



            new_simulated_moments = simulate_new_moments(p, S, N, game_parameters, simulation_parameters, data1, data2, A1, A2, n1)

            D[:, p] = (simulated_moments - new_simulated_moments) ./ delta


        end


    end
    
    
    
    return D
end
    

derivative_moments (generic function with 1 method)

In [23]:
Derivative = derivative_moments(S, N, game_parameters, simulation_parameters, data1, data2, n1, aux)

227.226370 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	178.121535 seconds (575.54 M allocations: 12.754 GiB, 1.22% gc time)
, 18.91% gc time)
parameter 7[0.93, 9.48, 0.09, 0.96][49.0, 30.0, 3.1976999999999998]
parameter 6[0.93, 9.48, 0.09, 0.96][49.0, 29.0, 3.23]
parameter 5[0.93, 9.48, 0.09, 0.96][48.0, 30.0, 3.23]
190.808640 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	179.294523 seconds (575.54 M allocations: 12.754 GiB, 1.49% gc time)
, 1.19% gc time)
parameter 4[0.93, 9.48, 0.09, 0.9503999999999999][48.0, 30.0, 3.23]
212.133684 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	201.467756 seconds (575.54 M allocations: 12.754 GiB, 9.56% gc time)
, 8.89% gc time)
parameter 3[0.93, 9.48, 0.0891, 0.96][48.0, 30.0, 3.23]
189.300016 seconds (633.40 M allocations: 14.035 GiB      From worker 3:	177.332139 seconds (575.54 M allocations: 12.754 GiB, 1.12% gc time)
, 1.24% gc time)
parameter 2[0.93, 9.385200000000001, 0.09, 0.96][48.0, 30.0, 3.

8×7 Array{Float64,2}:
   3.70976  -0.644874   -21.6085     3.02036  …  -0.307611      0.0858687
   0.0       0.0          0.0        0.0          0.0           0.0
   4.99149  -0.216007   -23.7351    -1.67029     -0.0303217     0.308496
  42.7175    1.14915      2.52525   -6.01326     -0.0689394    -0.178253
  19.6266    1.50413    -33.759    -46.8697      -0.539904     -1.11351
   6.56945   0.611586    63.1491     5.14358  …  -0.107467     -0.200658
   4.8599    0.0120066    4.0913     1.5965      -0.000517378  -0.0786309
 -17.279    -0.341621     2.42621    1.47016      0.0218955    -0.0225637

In [24]:
Derivative[:,2]

8-element Array{Float64,1}:
 -0.6448739051931223
  0.0
 -0.2160071124595145
  1.1491497250990939
  1.5041330656219154
  0.6115857257822519
  0.01200656270729233
 -0.3416208581357861

In [25]:
using DelimitedFiles

writedlm( "Derivative-nocd50_1pct.csv",  Derivative, ',')